In [14]:
import re 
import os
import json
import shutil
import psycopg2
from bs4 import BeautifulSoup

from pyspark.sql import Row
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,IntegerType
from pyspark.sql.functions import udf,col,regexp_extract,regexp_replace

conf = SparkConf()  # create the configuration
conf.set("spark.jars", "postgresql-42.2.6.jar")

sc = SparkSession \
    .builder \
    .config("spark.driver.extraClassPath", "./postgresql-42.2.6.jar") \
    .appName("boucled") \
    .getOrCreate()

sqlsc = SQLContext(sc)

In [9]:
topics_df.show(5)

+--------------------+-------------+---------+-------+--------------------+--------------------+--------+
|                 _id|       author|mod_title|n_posts|               title|               topic|topic_id|
+--------------------+-------------+---------+-------+--------------------+--------------------+--------+
|6284d856dd104e9ff...|  Shadow_man1|        0|     18|Pourquoi les gens...|https://www.jeuxv...|69748048|
|6284d856dd104e9ff...|     Zoom2022|        0|     31|AYAAAA POST OU CA...|https://www.jeuxv...|69717756|
|6284d856dd104e9ff...| TrollLutteur|        0|     50|Ma cousine de 19 ...|https://www.jeuxv...|69747925|
|6284d856dd104e9ff...|MichellAngelo|        0|     79|Si SEULEMENT j'ét...|https://www.jeuxv...|69748029|
|6284d856dd104e9ff...| DistribPLS16|        0|      5|Y a AUCUN BOULOT ...|https://www.jeuxv...|69747929|
+--------------------+-------------+---------+-------+--------------------+--------------------+--------+
only showing top 5 rows



In [15]:
in_path  = "/usr/src/app/project-boucle/src/boucled_scrapers/spiders/out/topics"
out_path = "/usr/src/app/project-boucle/src/boucled_scrapers/spiders/out/topics/processed" 
topics_df = sqlsc.read.json(in_path)
#Relier date/heure du premier d'un id topic a l'entrée topic dans la postgres avec un 
#rownum
final_df = topics_df.select("author", "title", "mod_title", "topic_id")
final_df = final_df.withColumn("topic_id",final_df["topic_id"].cast(IntegerType()))
final_df.write.format('jdbc').options(
  url='jdbc:postgresql://localhost:5432/boucled',
  driver='org.postgresql.Driver',
  dbtable='topics',
  user='postgres',
  password='password').mode('append').save()

conn = psycopg2.connect(dbname="boucled", user="postgres",
                        password="password", host="localhost")
conn.autocommit = True 
cur = conn.cursor()
cur.execute("""
DELETE FROM topics
WHERE pk_id IN
    (SELECT pk_id
    FROM
        (SELECT pk_id,
        ROW_NUMBER() OVER(PARTITION BY title 
        ORDER BY pk_id) AS row_num
        FROM topics) temp_table
        WHERE temp_table.row_num > 1);
""")
cur.close()
conn.close()

In [ ]:
out_dir = os.listdir(in_path)
for file_name in out_dir:
    shutil.move(os.path.join(in_path, file_name), out_path)